In [69]:
import tensorflow as tf
from keras.layers import LSTM, Dense, Input
import pandas as pd
import matplotlib.pyplot as plt

In [70]:
csv_file = "./climate.csv"
df = pd.read_csv(csv_file, sep=',')
data = df.iloc[:,[2]].values
# data = df.values

data

array([[-8.02],
       [-8.41],
       [-8.51],
       ...,
       [-3.16],
       [-4.23],
       [-4.82]])

Chuan hoa du lieu mean = 0, std = 1

In [71]:
def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

Chuan hoa va tach du lieu huan luyen

In [72]:
train_split = int(0.75 * int(df.shape[0]))
data = normalize(data, train_split)

train_data = data[:train_split]
val_data = data[train_split:]

Xac dinh phuong thuc du bao

In [73]:
past = 720
future = 72
step = 6
batch_size = 256



In [74]:
start = past + future
end = start + train_split

x_train = train_data
y_train = data[start:end]
sequence_length = int(past / step)

dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(x_train, y_train, sequence_length=sequence_length, sampling_rate=step, batch_size=batch_size)

In [75]:
x_end = len(val_data) - past - future
label_start = train_split + past + future
x_val = val_data[:x_end]
y_val = data[label_start:]
dataset_val = tf.keras.preprocessing.timeseries_dataset_from_array(x_val,y_val,sequence_length=sequence_length,sampling_rate=step,batch_size=batch_size)

In [76]:
model = tf.keras.Sequential()
model.add(Input(shape=(sequence_length, 1)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss="mse")
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [77]:
# Huấn luyện
epochs = 10
history = model.fit(
dataset_train,
epochs=epochs,
validation_data=dataset_val,
#callbacks=[es_callback, modelckpt_callback],
)

Epoch 1/10
1229/1230 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.8933

KeyboardInterrupt: 

In [ ]:

def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
visualize_loss(history, "Training and Validation Loss")